In [ ]:
import torch
from diffusers import StableDiffusion3ControlNetPipeline, SD3ControlNetModel
from diffusers.utils import load_image
from diffusers.image_processor import VaeImageProcessor

: 

In [ ]:
class SD3CannyImageProcessor(VaeImageProcessor):
    def __init__(self):
        super().__init__(do_normalize=False)
        
    def preprocess(self, image, **kwargs):
        image = super().preprocess(image, **kwargs)
        image = image * 255 * 0.5 + 0.5
        return image
    
    def postprocess(self, image, do_denormalize=True, **kwargs):
        do_denormalize = [True] * image.shape[0]
        image = super().postprocess(image, **kwargs, do_denormalize=do_denormalize)
        return image

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
controlnet = SD3ControlNetModel.from_pretrained(
        "stabilityai/stable-diffusion-3.5-large-controlnet-canny",
        torch_dtype=torch.float16
    ).to(device)
    
pipe = StableDiffusion3ControlNetPipeline.from_pretrained(
    "stabilityai/stable-diffusion-3.5-large",
    controlnet=controlnet,
    torch_dtype=torch.float16
).to(device)
pipe.image_processor = SD3CannyImageProcessor()